In [28]:
from qdrant_client import QdrantClient, models
import requests
import logging
from pprint import pprint
from fastembed import TextEmbedding
import pandas as pd
import random
import uuid

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [3]:
client = QdrantClient("http://localhost:6333")

In [4]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("__name__")

In [5]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'

try:
    response = requests.get(docs_url)
except Exception as e:
    logger.error("Failed to fetch docs due to: %s", e)
docs_json = response.json()

In [24]:
pprint(docs_json[0])

{'course': 'data-engineering-zoomcamp',
 'documents': [{'question': 'Course - When will the course start?',
                'section': 'General course-related questions',
                'text': 'The purpose of this document is to capture frequently '
                        'asked technical questions\n'
                        'The exact day and hour of the course will be 15th Jan '
                        '2024 at 17h00. The course will start with the first  '
                        "“Office Hours'' live.1\n"
                        'Subscribe to course public Google Calendar (it works '
                        'from Desktop only).\n'
                        'Register before the course starts using this link.\n'
                        'Join the course Telegram channel with announcements.\n'
                        "Don’t forget to register in DataTalks.Club's Slack "
                        'and join the channel.'},
               {'question': 'Course - What are the prerequisites f

In [7]:
pd.DataFrame(TextEmbedding.list_supported_models())

,model,sources,model_file,description,license,size_in_GB,additional_files,dim,tasks
0,BAAI/bge-base-en,"{'hf': 'Qdrant/fast-bge-base-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en.tar.gz', '_deprecated_tar_struct': True}",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",mit,0.420,[],768,{}
1,BAAI/bge-base-en-v1.5,"{'hf': 'qdrant/bge-base-en-v1.5-onnx-q', 'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-base-en-v1.5.tar.gz', '_deprecated_tar_struct': True}",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",mit,0.210,[],768,{}
2,BAAI/bge-large-en-v1.5,"{'hf': 'qdrant/bge-large-en-v1.5-onnx', 'url': None, '_deprecated_tar_struct': False}",model.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",mit,1.200,[],1024,{}
3,BAAI/bge-small-en,"{'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.",mit,0.130,[],384,{}
4,BAAI/bge-small-en-v1.5,"{'hf': 'qdrant/bge-small-en-v1.5-onnx-q', 'url': None, '_deprecated_tar_struct': False}",model_optimized.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",mit,0.067,[],384,{}
5,BAAI/bge-small-zh-v1.5,"{'hf': 'Qdrant/bge-small-zh-v1.5', 'url': 'https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz', '_deprecated_tar_struct': True}",model_optimized.onnx,"Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",mit,0.090,[],512,{}
6,mixedbread-ai/mxbai-embed-large-v1,"{'hf': 'mixedbread-ai/mxbai-embed-large-v1', 'url': None, '_deprecated_tar_struct': False}",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year.",apache-2.0,0.640,[],1024,{}
7,snowflake/snowflake-arctic-embed-xs,"{'hf': 'snowflake/snowflake-arctic-embed-xs', 'url': None, '_deprecated_tar_struct': False}",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year.",apache-2.0,0.090,[],384,{}
8,snowflake/snowflake-arctic-embed-s,"{'hf': 'snowflake/snowflake-arctic-embed-s', 'url': None, '_deprecated_tar_struct': False}",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year.",apache-2.0,0.130,[],384,{}
9,snowflake/snowflake-arctic-embed-m,"{'hf': 'Snowflake/snowflake-arctic-embed-m', 'url': None, '_deprecated_tar_struct': False}",onnx/model.onnx,"Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2024 year.",apache-2.0,0.430,[],768,{}


# Dense search

In [8]:
model_name = 'jinaai/jina-embeddings-v2-small-en'

In [9]:
collection_name = 'zoomcamp-rag'
EMBEDDING_DIMENSIONALITY = 512

collection_created = client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

print(f"Collection created: {collection_created}")

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-rag "HTTP/1.1 409 Conflict"


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `zoomcamp-rag` already exists!"},"time":0.000405334}'

In [10]:
points = []
id = 0

for course in docs_json:
    for doc in course['documents']:
        new_point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_name),
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            }
        )
        points.append(new_point)
        id = id + 1

In [11]:
client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.43s/it]
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-rag/points?wait=true "HTTP/1.1 200 OK"


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
def search(query, limit=1):
    return client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name
        ),
        limit=limit,
        with_payload=True
    )

In [14]:
course = random.choice(docs_json)
course_piece = random.choice(course['documents'])
pprint(course_piece, sort_dicts=False, indent=2)

{ 'text': 'When the request is processed in lambda function, mlflow library '
          'raises:\n'
          '2022/09/19 21:18:47 WARNING mlflow.pyfunc: Encountered an '
          'unexpected error (AttributeError("module \'dataclasses\' has no '
          'attribute \'__version__\'")) while detecting model dependency '
          'mismatches. Set logging level to DEBUG to see the full traceback.\n'
          'Solution: Increase the memory of the lambda function.\n'
          'Added by MarcosMJD',
  'section': 'Module 4: Deployment',
  'question': 'Error raised when executing mlflow’s pyfunc.load_model in '
              'lambda function.'}


In [18]:
pprint(search(course_piece['question']), indent=2, sort_dicts=False)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-rag/points/query "HTTP/1.1 200 OK"


QueryResponse(points=[ScoredPoint(id=912, version=0, score=0.917132, payload={'text': 'When the request is processed in lambda function, mlflow library raises:\n2022/09/19 21:18:47 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module \'dataclasses\' has no attribute \'__version__\'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.\nSolution: Increase the memory of the lambda function.\nAdded by MarcosMJD', 'section': 'Module 4: Deployment', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])


In [19]:
print(search("What if I submit homeworks late?").points[0].payload['text'])

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-rag/points/query "HTTP/1.1 200 OK"


No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [20]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-rag/index?wait=true "HTTP/1.1 200 OK"


UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [21]:
def search_in_course(query, course, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_name
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )
    return results

In [26]:
for course in [
    "mlops-zoomcamp", 
    "machine-learning-zoomcamp", 
    "data-engineering-zoomcamp"
]:
    pprint(search_in_course('Can I submit my homework late?', course, 1))

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-rag/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-rag/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-rag/points/query "HTTP/1.1 200 OK"


QueryResponse(points=[ScoredPoint(id=814, version=0, score=0.77857745, payload={'text': 'Please choose the closest one to your answer. Also do not post your answer in the course slack channel.', 'section': '+-General course questions', 'course': 'mlops-zoomcamp'}, vector=None, shard_key=None, order_value=None)])
QueryResponse(points=[ScoredPoint(id=451, version=0, score=0.85317105, payload={'text': 'No, it’s not possible. The form is closed after the due date. But don’t worry, homework is not mandatory for finishing the course.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])
QueryResponse(points=[ScoredPoint(id=15, version=0, score=0.9226917, payload={'text': 'No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y\nOlder news:[source1] [source2]', 's

# Sparse search

In [27]:
collection_created = client.create_collection(
    collection_name="zoomcamp-sparse",
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(
            modifier=models.Modifier.IDF,
        )
    }
)

print(f"Collection created: {collection_created}")

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-sparse "HTTP/1.1 200 OK"


Collection created: True


In [30]:
client.upsert(
    collection_name="zoomcamp-sparse",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "bm25": models.Document(
                    text=doc["text"],
                    model="Qdrant/bm25",
                ),
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
    for course in docs_json
    for doc in course["documents"]
    ]
)

Fetching 18 files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 26.88it/s]
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-sparse/points?wait=true "HTTP/1.1 200 OK"


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
def search(query: str, limit: int = 1) -> list[models.ScoredPoint]:
    results = client.query_points(
        collection_name="zoomcamp-sparse",
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )

    return results.points

In [42]:
search(course_piece['question'])[0].payload["text"]

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-sparse/points/query "HTTP/1.1 200 OK"


'When the request is processed in lambda function, mlflow library raises:\n2022/09/19 21:18:47 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module \'dataclasses\' has no attribute \'__version__\'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.\nSolution: Increase the memory of the lambda function.\nAdded by MarcosMJD'

# Hybrid search

In [44]:
client.create_collection(
    collection_name="zoomcamp-hybrid",
    vectors_config={
        "jina-small": models.VectorParams(size=512, distance=models.Distance.COSINE)
    },
    sparse_vectors_config={
        "bm25": models.SparseVectorParams(modifier=models.Modifier.IDF)
    }
)

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-hybrid "HTTP/1.1 200 OK"


True

In [48]:
client.upsert(
    collection_name="zoomcamp-hybrid",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                "jina-small": models.Document(
                    text=doc["text"],
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                "bm25": models.Document(
                    text=doc["text"],
                    model="Qdrant/bm25",
                )
            },
            payload={
                "text": doc["text"],
                "section": doc["section"],
                "course": course["course"],
            }
        )
        for course in docs_json
        for doc in course["documents"]
    ]
)

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/zoomcamp-hybrid/points?wait=true "HTTP/1.1 200 OK"


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [49]:
def hybrid_search(query: str, limit: int=1):
    results = client.query_points(
        collection_name="zoomcamp-hybrid",
        prefetch=[
            models.Prefetch(
                query=models.Document(
                    text=query,
                    model="jinaai/jina-embeddings-v2-small-en",
                ),
                using="jina-small",
                limit=10*limit,
            ),
        ],
        query=models.Document(
            text=query,
            model="Qdrant/bm25",
        ),
        using="bm25",
        limit=limit,
        with_payload=True,
    )
    return results.points

In [50]:
pprint(course_piece)

{'question': 'Error raised when executing mlflow’s pyfunc.load_model in lambda '
             'function.',
 'section': 'Module 4: Deployment',
 'text': 'When the request is processed in lambda function, mlflow library '
         'raises:\n'
         '2022/09/19 21:18:47 WARNING mlflow.pyfunc: Encountered an unexpected '
         'error (AttributeError("module \'dataclasses\' has no attribute '
         '\'__version__\'")) while detecting model dependency mismatches. Set '
         'logging level to DEBUG to see the full traceback.\n'
         'Solution: Increase the memory of the lambda function.\n'
         'Added by MarcosMJD'}


In [51]:
results=hybrid_search(course_piece["question"])
print(results[0].payload["text"])

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/zoomcamp-hybrid/points/query "HTTP/1.1 200 OK"


When the request is processed in lambda function, mlflow library raises:
2022/09/19 21:18:47 WARNING mlflow.pyfunc: Encountered an unexpected error (AttributeError("module 'dataclasses' has no attribute '__version__'")) while detecting model dependency mismatches. Set logging level to DEBUG to see the full traceback.
Solution: Increase the memory of the lambda function.
Added by MarcosMJD
